<a href="https://colab.research.google.com/github/anelglvz/Working-Analyst/blob/main/Matem%C3%A1ticas_CD/Calculo_GradienteDescendiente_Regresi%C3%B3nLog%C3%ADstica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción

Se verá como funciona el algoritmo de Gradiente Descendiente aplicado a una regresión logística que asignará las probabilidades utilizando la función softmax.

# Dependencias

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Solo para que al mostrarnos los arreglos no nos de los valores grandes en notación científica
np.set_printoptions(suppress=True)

# Carga de datos e interpretación


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Curso-Algebra/semana10/pruebas_datos.csv')

In [ ]:
df

In [ ]:
# Los valores 1 son NON-HFT y los 0 son HFT
df.type.value_counts()

## Agregando descripción a los datos

Los datos son públicos, fueron utilizados en una competencia que pueden consultar [AQUÍ](https://challengedata.ens.fr/participants/challenges/50/) (para poder ingresar necesitarán registrarse y luego copiar la dirección en su navegador). En esta competencia registraron muchas características de "traders", para tratar de clasificarlos en HFT (High Frequency Traders) o NON-HFT, esto porque es de gran interes tratar de estudiar el impacto que tienen los HFT en los mercados, ya que lo que hacen es realizar intercambios dando ordenes de intercambio a un mecanismo llamado Limit Order Book (LOB), al cual le ordenan realizar compras o ventas a precios que ellos deseen.

## Descripción de las variables
1. NbTradeVenueMic (23) : Número de centros de negociación en los que opera el jugador del mercado

De todos los centros de negociación, estadísticas sobre el número de operaciones observadas por segundo

2. MaxNbTradesBySecond (24)
3. MeanNbTradesBySecond (25) 

Estadísticas sobre el delta de tiempo observado entre dos operaciones en la plataforma de negociación TV_1[1]:

4. min_dt_TV1 (26)
5. med_dt_TV1 (28)
6. mean_dt_TV1 (27)

Estadísticas sobre el delta de tiempo observado entre dos operaciones que se producen en la plataforma de negociación TV_1 y luego en la plataforma de negociación TV_2:

7. min_dt_TV1_TV2 (29)
8. med_dt_TV1_TV2 (31)
9. mean_dt_TV1_TV2 (30)

Estadísticas sobre el delta de tiempo observado entre dos operaciones que se producen en la plataforma de negociación TV_1 y luego en la plataforma de negociación TV_3:

10. min_dt_TV1_TV3 (32)
11. med_dt_TV1_TV3 (34)
12. mean_dt_TV1_TV3 (33)

Estadísticas sobre el delta de tiempo observado entre dos operaciones que se producen en la plataforma de negociación TV_1 y luego en la plataforma de negociación TV_4 :

13. min_dt_TV1_TV4 (35)
14. med_dt_TV1_TV4 (37)
15. mean_dt_TV1_TV4(36)

De todos los centros de negociación, número de segundos durante el día de negociación donde se observa al menos una operación del jugador del mercado i

16. NbSecondWithAtLeatOneTrade(38)

En la plataforma de negociación TV_1, tres proporciones entre el número de todo tipo de eventos[2] enviados a la LOB y:

17. el número de operaciones (OTR) (4)
18. el número de eventos de tipo cancelación (OCR) (5)
19. el número de eventos de tipo de modificación (OMR) (6)

En el centro de negociación TV_1, estadísticas durante el delta de tiempo observado entre dos eventos de todo tipo enviados:

20. min_time_two_events (7)
21. mean_time_two_events (8)
22. 10_p_time_two_events (9)
23. med_time_two_events (10)
24. 25_p_time_two_events (11)
25. 75_p_time_two_events (12)
26. 90_p_time_two_events (13)
27. max_time_two_events (14)

En la plataforma de negociación TV_1, estadísticas a lo largo de la vida útil observada de las órdenes canceladas:

28. min_lifetime_cancel (15)
29. mean_lifetime_cancel (16)
30. 10_p_lifetime_cancel (17)
31. med_lifetime_cancel (18)
32. 25_p_lifetime_cancel (19)
33. 75_p_lifetime_cancel (20)
34. 90_p_lifetime_cancel (21)
35. max_lifetime_cancel (22)

Extra variables

36. Type: HF, Non HFT (39)
37. Day
38. Share
39. Trader
40. Index


In [ ]:
df.shape

In [ ]:
X_prob = np.array(df.drop(['type'], axis=1))
X_prob

In [ ]:
X_prob.shape

In [ ]:
Y_prob = np.array(df['type'])
Y_prob

In [ ]:
Y_prob.shape

In [ ]:
num_inputs = 25
num_outputs = 2

W = np.random.normal(0, 0.1, (num_inputs, num_outputs)) # Jugaremos con este parametro
B = np.zeros(num_outputs) # Jugaremos con este parametro

In [ ]:
print(W)
print(B)

# Definición de la operación SoftMax

Primero, notemos que dado un arreglo de numpy, nosotros podemos relizar sumas de diferentes formas utilizando el método "sum()". Al daro distintos argumentos se reliza de distintos modos.

Si no pasamos ningún parámetro a "sum()", lo que obtendremos será la suma de todas las entradas de nuestra matriz (o arreglo o tensor). 

Si damos como parámetro el "0", lo que nos regresará será la suma por columnas (es decir, en el eje 0)

Si damos como parámetro el "1", nos regresará la suma por filas (es decir, en el eje 1)

In [ ]:
X = np.array([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
X

In [ ]:
X.sum(keepdims=True), X.sum(0, keepdims=True), X.sum(1, keepdims=True)

Con esto podemos implementar de modo muy sencillo la función SoftMax, que consiste de 3 pasos:

1. Obtener el exponenete de cda uno de nuestros elementos.
2. Tomamos la suma de los elementos de su fila
3. Dividimos cada fila por su constante de normalización (sería una suma de exponenciales)

La expresión matemática de la función SoftMax es la siguiente:


$$ \mathrm{softmax}(\mathbf{X})_{ij} = \frac{\exp(\mathbf{X}_{ij})}{\sum_k \exp(\mathbf{X}_{ik})}. $$

In [ ]:
def softmax(X):
    X_exp = np.exp(X)
    partition = X_exp.sum(1, keepdims=True)
    return X_exp / partition

Veamos un ejemplo con la matriz de 2X3 creada anteriormente.

In [ ]:
softmax(X)

Si observan con cuidado, pueden ver que con esta operación estamos transformando todas nuestras entradas en números positivos. Mas aún, cada fila suma 1, es decir, la convertimos en una distribución de probabilidad.

In [ ]:
x = np.random.normal(0, 1, (2, 5))
x_prob = softmax(x)
x_prob, x_prob.sum(1)

# Definiendo el modelo

Aquí pueden ver que tipo de modelo vamos a implementar. En este caso es una regresión softmax.

In [ ]:
def net(X,W,b):
    return softmax(np.dot(X.reshape((-1, W.shape[0])), W) + b)

1. ¿Que nos dará como salida ésta operación?

2. ¿Porqué?

# Definiendo la función de perdida

Usaremos la función Cross-Entropy, una de las mas populares dado que es buena para problemas de clasficación y en la actualidad, en machine learning son mas los problemas de clasificación que los de regresión.

La función es la siguiente:
$$ -\sum_{c=1}^My_{o,c}\log(p_{o,c}), $$

donde $M$ es la cantidad de clases que tenemos, $log$ la función de logaritmo natural, $y$ es $0$ o $1$ dependiendo de la etiqueta real de la clase $c$ y por último $p_{o,c}$ es la probabilidad predicha de que la observación $o$ pertenezca a la clase $c$.

Para calcular esto, en lugar de utilizar un ciclo "for" (que a veces termina siendo menos optimizado), utilizaremos funciones de numpy.

In [ ]:
def cross_entropy(y_hat, y):
    return np.sum(- np.log(y_hat[range(len(y_hat)), y])) / y.shape[0]

In [ ]:
# El resultado final es lo que alimentará a la cross-entropy, es decir etiqueta 
# de que pertenece a la clase 1 por la probabilidad de pertenecer a la clase 1
y = np.array([0, 2])
y_hat = np.array([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y_hat[[0, 1], y]

In [ ]:
cross_entropy_each(y_hat, y)

# Precisión de la clasificación

En un problema de clasificación multi-clase, es decir, un problema en el que observemos varias caracteristicas de un elemento y decidamos a que clase pertenece dicho elemento. 

Por ejemplo: Un algoritmo de clasificación de imagenes que intente saber si algo es un perro, un gato o un ratón. Si introdujeramos características y al final diera un vector $(0.2, 0.38, 0.42)$, el algoritmo lo clasificaría como un ratón (lo marcaría como perteneciente a la tercer clase)

Cuando las predicciones son consistentes con las etiquetas verdaderas, decimos que son correctas. La presición de la clasificación son la cantidad de predicciones correctas entre todas las predicciones. Sin embargo, es muy difícil optimizar esto directamente (no es una función diferenciable), pero será muy importante para ver el rendimiento de nuestro modelo.

Lo que hacemos para ver la precisión es encontrar el valor mas grande en el vector de probabilidades, convertirlo en entero (porque muy probablemente sea un valor entr 0 y 1) y compararlo con las etiquetas verdaderas de nuestros datos.

In [ ]:
# ARGMAX regresa la posicion del elemento con mayor valor
np.array([[0.2,0.38, 0.42]]).argmax(axis=1)

In [ ]:
def accuracy(y_hat, y):
    """Calcula la cantidad de predicciones correctas."""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.astype(y.dtype) == y
    return float(cmp.astype(y.dtype).sum())/ len(y)

In [ ]:
accuracy(y_hat, y) 

## Veamos como sería el accuracy para nuestro modelo con pesos al azar

In [ ]:
X_prob

In [ ]:
accuracy(net(X_prob, W, B),Y_prob) 

In [ ]:
W = np.random.normal(0, 0.1, (num_inputs, num_outputs)) # Jugaremos con este parametro
B = np.zeros(num_outputs) # Jugaremos con este parametro

In [ ]:
# Las probabilidades que predecimos para cada fila de nuestros datos
net(X_prob, W, B)

In [ ]:
# La etiqueta verdadera de nuestros datos
Y_prob

# SoftMax Estable


In [ ]:
# El resultado final es lo que alimentará a la cross-entropy, es decir etiqueta 
# de que pertenece a la clase 1 por la probabilidad de pertenecer a la clase 1
y = np.array([0, 2])
y_hat = np.array([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y_hat[[0, 1], y]

In [ ]:
# Función softmax (Repetida de arriba solo para ejemplificar)
def softmax(X):
    X_exp = np.exp(X)
    partition = X_exp.sum(1, keepdims=True)
    return X_exp / partition

In [ ]:
# Softmax Estable
def stable_softmax(X):
    X_exp = np.exp(X - np.max(X, axis=1, keepdims=True))
    partition = X_exp.sum(1, keepdims=True)
    return X_exp / partition

In [ ]:
def net_s(X,W,b):
    return stable_softmax(np.dot(X.reshape((-1, W.shape[0])), W) + b)

# Demostración de SoftMax estable

In [ ]:
prueba = np.array([[99999, 99995, 99993],[9, 5, 3]])

In [ ]:
softmax(prueba)

In [ ]:
stable_softmax(prueba)

In [ ]:
# Si está repetida, es solo para recordar su estructura
def cross_entropy(y_hat, y):
    return np.sum(- np.log(y_hat[range(len(y_hat)), y])) / y.shape[0]

In [ ]:
cross_entropy(y_hat, y)

¿Como seria el gradiente de mi función compuesta?

$$ \require{cancel} 
\begin{align} 
\frac{dL}{dz_i} &= \frac{d}{dz_i} \bigg[  – \sum_{k=1}^c y_k log (a_k) \bigg] \\ 
&= – \sum_{k=1}^c y_k \frac{d \big( log (a_k) \big)}{dz_i} \\ 
&= – \sum_{k=1}^c y_k \frac{d \big( log (a_k) \big)}{da_k} . \frac{da_k}{dz_i} \\ 
&= – \sum_{k=1}^c\frac{y_k}{a_k} . \frac{da_k}{dz_i} \\ 
&= – \bigg[ \frac{y_i}{a_i} . \frac{da_i}{dz_i}  + \sum_{k=1, k \not=i}^c \frac{y_k}{a_k}  \frac{da_k}{dz_i} \bigg] \\ 
&= – \frac{y_i}{\cancel{a_i}} . \cancel{a_i}(1-a_i) \text{ } – \sum_{k=1, k \not=i}^c \frac{y_k}{\cancel{a_k}} . (\cancel{a_k}a_i) \\ 
&= – y_i +y_ia_i + \sum_{k=1, k \not=i}^c y_ka_i \\ 
&= a_i \big( y_i + \sum_{k=1, k \not=i}^c y_k \big) – y_i \\ 
&= a_i + \sum_{k=1}^c y_k -y_i \\ 
&= a_i . 1 – y_i \text{ , since } \sum_{k=1}^c y_k =1 \\ 
&= a_i – y_i 
\end{align} $$

In [ ]:
def grad_softmax_cross_entropy_for_w(X, y_true, w, b):
    netaux = net_s(X,w,b)

    ones_true_class = np.zeros_like(netaux) # Matriz de ceros que tiene la forma de las probabilidades (logits.shape), si alimento todos los datos es 50000X2

    ones_true_class[np.arange(len(netaux)),y_true] = 1 # Genera 1 o cero si esta en la clasificacion correcta
 
    softmax_ = netaux # softmax aplicado a mis probabilidades

    aux1 = softmax_ - ones_true_class

    return X.T @ aux1/X.shape[0]

In [ ]:
def grad_softmax_cross_entropy_for_b(X, y_true, w, b):
    netaux = net_s(X,w,b)

    ones_true_class = np.zeros_like(netaux) # Matriz de ceros que tiene la forma de las probabilidades (logits.shape), si alimento todos los datos es 50000X2

    ones_true_class[np.arange(len(netaux)),y_true] = 1 # Genera 1 o cero si esta en la clasificacion correcta
 
    softmax_ = netaux # softmax aplicado a mis probabilidades

    aux1 = softmax_ - ones_true_class

    return aux1.sum(axis=0, keepdims=True)/X.shape[0]

Ahora podemos implementar Gradient Descent

In [ ]:
def gradient_descent(x, y, w, b, net, learning_rate=0.1, num_epochs=10):
	m = x.shape[0]
	J_all = []
	
	for _ in range(num_epochs):
		y_probs = net_s(x, w, b)
		y_hat = y_probs.argmax(axis=1)
		
		gradient_w = grad_softmax_cross_entropy_for_w(x,y,w,b)
		gradient_b = grad_softmax_cross_entropy_for_b(x,y,w,b)
   
	 #Actualización de los parámetros
		w = w - (learning_rate)*gradient_w
		b = b - (learning_rate)*gradient_b
		
		J_all.append(cross_entropy(y_probs, y))

	return J_all, w, b

In [ ]:
W = np.random.normal(0, 0.1, (num_inputs, num_outputs)) # Jugaremos con este parametro
B = np.zeros(num_outputs) # Jugaremos con este parametro

In [ ]:
learning_rate = 0.01
num_epochs = 1000

In [ ]:
J_all, W, B = gradient_descent(X_prob, Y_prob, W, B, net, learning_rate, num_epochs)
Y_hat = net(X_prob, W, B)
J = cross_entropy(Y_hat, Y_prob)
print("Cost: ", J)
print("Parameters: ", W)
print("Intercepto: ", B)

In [ ]:
print(J_all)

In [ ]:
perdida_por_epoca = np.array(J_all).reshape(-1,1)
plt.plot(perdida_por_epoca)
plt.show()

In [ ]:
# Las probabilidades que predecimos para cada fila de nuestros datos
net(X_prob, W, B)

In [ ]:
# La etiqueta verdadera de nuestros datos
Y_prob

In [ ]:
accuracy(net(X_prob, W, B),Y_prob) 

# SGD: Stochastic Gradient Descent

En este caso, el algoritmo entrenara de modo un poco distinto, tomando muestras al azar de mis datos, dicha muestra será del tamaño que yo la pida.

In [ ]:
batch_size = 200

In [ ]:
number_of_rows = X_prob.shape[0]  # En nuestro caso, son 50,000
random_indices = np.random.choice(number_of_rows, size=batch_size, replace=False)
X_prima = X_prob[random_indices, :].copy()

In [ ]:
X_prima

In [ ]:
Y_prima = Y_prob[random_indices].copy()

In [ ]:
Y_prima

In [ ]:
def batch_gradient_descent(x, y, w, b, net, batch_size, learning_rate=0.1, num_epochs=10):
  m = x.shape[0]
  J_all = []
	
  for _ in range(num_epochs):
    random_indices = np.random.choice(m, size=batch_size, replace=False)
    x_batch = x[random_indices, :].copy()
    y_batch = y[random_indices].copy()

    y_probs = net_s(x_batch, w, b)
    #y_hat = y_probs.argmax(axis=1)
    
    gradient_w = grad_softmax_cross_entropy_for_w(x_batch,y_batch,w,b)
    gradient_b = grad_softmax_cross_entropy_for_b(x_batch,y_batch,w,b)

    #Actualizamos los paramentros
    w = w - (learning_rate)*gradient_w
    b = b - (learning_rate)*gradient_b
    
    J_all.append(cross_entropy(y_probs, y_batch))

  return J_all, w, b 

In [ ]:
W = np.random.normal(0, 0.1, (num_inputs, num_outputs)) # Jugaremos con este parametro
B = np.zeros(num_outputs) # Jugaremos con este parametro

In [ ]:
learning_rate = 0.01
num_epochs = 1000

In [ ]:
J_all, W, B = batch_gradient_descent(X_prob, Y_prob, W, B, net, batch_size, learning_rate, num_epochs)
Y_hat = net(X_prob, W, B)
J = cross_entropy(Y_hat, Y_prob)
print("Cost: ", J)
print("Parameters: ", W)
print("Intercepto: ", B)

In [ ]:
print(J_all)

In [ ]:
perdida_por_epoca = np.array(J_all).reshape(-1,1)
plt.plot(perdida_por_epoca)
plt.show()

In [ ]:
# Las probabilidades que predecimos para cada fila de nuestros datos
net(X_prob, W, B)

In [ ]:
# La etiqueta verdadera de nuestros datos
Y_prob

In [ ]:
accuracy(net(X_prob, W, B),Y_prob) 